# Building Location-Aware Agents with LangChain & Camino AI 🗺️🤖

Welcome to this comprehensive tutorial on building intelligent location-aware agents using **LangChain** and **Camino AI**. While web search tools like Tavily help agents access online information, Camino AI enables agents to understand and reason about the physical world through location intelligence.

## What You'll Learn

In this tutorial, you'll discover how to:
- 🔧 **Set up Camino AI tools** with LangChain for location intelligence
- 🤖 **Build autonomous agents** that can search places, plan routes, and understand spatial context
- 🧠 **Create ReAct agents** that reason about location-based tasks
- 🔗 **Combine multiple tools** for complex real-world scenarios
- 💾 **Add memory** to agents for contextual location awareness
- 🚀 **Deploy production-ready** location-aware AI systems

## Why Camino AI + LangChain?

The combination of Camino AI and LangChain provides:
- **Natural Language Understanding**: Convert queries like "quiet cafes for working" into actionable location searches
- **Spatial Reasoning**: Calculate distances, routes, and relationships between places
- **Context Awareness**: Understand neighborhoods, amenities, and local context
- **Journey Planning**: Optimize multi-stop trips with constraints
- **Cost-Effective**: $0.001 per API call
- **Agent-First Design**: Built specifically for AI agents, not traditional apps

---

## Part 1: Setup and Installation

### Prerequisites

1. **Sign up for Camino AI** at [app.getcamino.ai](https://app.getcamino.ai?utm_source=agents-towards-production&utm_medium=github&utm_campaign=tutorial) to get your API key
2. **Get an Anthropic API key** (or use another LLM provider supported by LangChain)
3. **Install dependencies** using the cells below

In [ ]:
# Install required packages
%pip install --upgrade langchain langchain-anthropic langchain-community langgraph langchain-camino python-dotenv --quiet

### Initialize API Clients

In [1]:
import os
import getpass
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Prompt for API keys if not set
if not os.environ.get("CAMINO_API_KEY"):
    os.environ["CAMINO_API_KEY"] = getpass.getpass("Enter your Camino API Key: ")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API Key: ")

print("✅ API keys configured successfully!")

Enter your Camino API Key:  ········
Enter your Anthropic API Key:  ········


✅ API keys configured successfully!


## Part 2: Setting Up Camino AI Tools

Camino AI provides six main tools for LangChain:
1. **CaminoSearch**: Natural language place search with AI ranking
2. **CaminoQuery**: Advanced queries with context and spatial reasoning
3. **CaminoRoute**: Routing between two points
4. **CaminoJourney**: Multi-stop journey planning and optimization
5. **CaminoContext**: Provides an Agent-ready contextual overview of a location
6. **CaminoRelationship**: Provides the relationship between two distinct points

In [6]:
# Import Camino tools for LangChain
from langchain_camino import CaminoSearch, CaminoQuery, CaminoJourney
            
camino_search = CaminoSearch()
camino_query = CaminoQuery()
camino_journey = CaminoJourney()

print("🛠️ Camino tools initialized:")
print(f"  • {camino_search.name}: {camino_search.description}")
print(f"  • {camino_query.name}: {camino_query.description}")
print(f"  • {camino_journey.name}: {camino_journey.description}")


🛠️ Camino tools initialized:
  • camino_search: Search for places by name using the Camino AI Reality Grounding API. This tool uses Nominatim for location lookup and returns detailed place information including coordinates, addresses, and place types. Useful for finding specific locations, businesses, landmarks, or points of interest.
  • camino_query: Make natural language location queries using the Camino AI Reality Grounding API. This tool converts natural language queries to location searches and supports location-based POI discovery with optional AI-powered result ranking. Useful for finding places based on descriptive queries like 'coffee shops near me', 'best restaurants in downtown', or 'parks within walking distance'.
  • camino_journey: Plan multi-waypoint journeys using the Camino AI Reality Grounding API. This tool provides route optimization, supports different transport modes, and handles complex journey planning with multiple stops. Useful for planning trips with multipl

### Test Individual Tools

In [8]:
# Test CaminoSearch tool
print("🔍 Testing CaminoSearch...\n")

result = camino_search.invoke({"query": "eiffel tower", "limit": 1})

print(result)

print("🔍 Testing CaminoQuery...\n")
result = camino_query.invoke({"query":"cafes in Brooklyn NY"})

print(result)

🔍 Testing CaminoSearch...

{'places': [{'name': 'Eiffel Tower', 'lat': 43.2844294, 'lon': 28.0440305, 'address': '', 'place_type': ''}], 'query': 'eiffel tower', 'total_results': 7}
🔍 Testing CaminoQuery...

{'results': [{'name': 'La Fondita', 'lat': 40.6647223, 'lon': -73.9895351, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 1.0}, {'name': 'Mocha Hookah', 'lat': 40.6901914, 'lon': -73.9935675, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.9}, {'name': 'Blank Street Coffee', 'lat': 40.6872881, 'lon': -73.9753082, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.8}, {'name': 'Southside Coffee', 'lat': 40.6611342, 'lon': -73.9902727, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.7}, {'name': 'Doughnut Plant', 'lat': 40.7174274, 'lon': -73.9583326, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.6}, {'name': 'Canteen Delicatessen & Cafe', 'lat': 40.6825167, 'lon': -73.9793167, 'distance': None, '

In [19]:
# Test CaminoQuery for advanced context-aware search
print("🎯 Testing CaminoQuery with included answer\n")

query_result = camino_query.invoke({
    "query": "cafe",
    "lat": 40.779144,  # Central Park area
    "lon": -73.962429,
    "radius": 500,
    "answer": True,
    "rank": True
})

print(f"💡 Recommendation: {query_result}\n")

🎯 Testing CaminoQuery with included answer

💡 Recommendation: {'results': [{'name': 'City Market Cafe', 'lat': 40.7786106, 'lon': -73.9606629, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 1.0}, {'name': 'Starbucks', 'lat': 40.7799038, 'lon': -73.9597037, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.9}, {'name': 'Tal Bagels', 'lat': 40.7777126, 'lon': -73.9570936, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.8}, {'name': 'American Wing Café', 'lat': 40.7802595, 'lon': -73.9635992, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.7}, {'name': 'Petrie Court Café', 'lat': 40.7791519, 'lon': -73.9644503, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.6}, {'name': 'E.A.T.', 'lat': 40.7773359, 'lon': -73.9615905, 'distance': None, 'type': 'cafe', 'rating': None, 'confidence': 0.5}, {'name': 'Dear Coffee', 'lat': 40.778152, 'lon': -73.9567392, 'distance': None, 'type': 'cafe', 'rating': None, 'conf

In [25]:
# Test CaminoJourney for trip planning
print("🗺️ Testing CaminoJourney...\n")

# journey_result = camino_journey.invoke({
#     "waypoints": [
#         {"lat":40.77903,"lon":-73.962353,"purpose":"Museum"},
#         {"lat":40.7786106,"lon":-73.9606629,"purpose":"Coffee Shop"}
#     ],
#     "constraints": {
#         "transport":"walking",
#         "time_budget":"30 minutes"
#     }
# })

journey_result = camino_journey.invoke({
                "waypoints": [(40.77903, -73.962353), (40.7786106, -73.9606629), (40.758745, -73.985123)],
                "transport_mode": "walk",
                "optimize": True
            })

print(journey_result)

🗺️ Testing CaminoJourney...

{'journey': {'total_distance': 3.17, 'total_duration': None, 'optimized': True, 'transport_mode': 'walk', 'feasible': True, 'waypoints': [{'lat': 40.77903, 'lon': -73.962353, 'order': 0, 'purpose': 'Stop 1'}, {'lat': 40.7786106, 'lon': -73.9606629, 'order': 1, 'purpose': 'Stop 2'}, {'lat': 40.758745, 'lon': -73.985123, 'order': 2, 'purpose': 'Stop 3'}], 'route_geometry': '', 'instructions': []}}


## Part 3: Building a ReAct Location Agent

Now let's build an intelligent agent that can reason about location-based tasks using the ReAct (Reasoning and Acting) framework.

In [26]:
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import datetime

# Initialize the LLM
llm = ChatAnthropic(
    model="claude-3-7-sonnet-latest",
    temperature=0.7
)

# Get today's date for context
today = datetime.datetime.today().strftime("%A, %B %d, %Y")

# Create the location-aware agent
location_agent = create_react_agent(
    model=llm,
    tools=[camino_search, camino_query, camino_journey],
    prompt=ChatPromptTemplate.from_messages([
        (
            "system",
            f"""
You are a highly capable location-aware AI assistant equipped with Camino AI tools for understanding and navigating the physical world. Your mission is to help users with location-based tasks, providing accurate, context-aware, and actionable information.

**Today's Date:** {today}

**Available Tools:**

1. **CaminoSearch** - Natural language place search
   - Use for: Finding specific types of places (restaurants, shops, services)
   - Features: AI-powered ranking, distance calculation, relevance scoring
   - Best for: "Find coffee shops near me" or "Where can I buy electronics?"

2. **CaminoQuery** - Advanced contextual search
   - Use for: Complex queries requiring context understanding
   - Features: Context-aware results, intent recognition, detailed summaries
   - Best for: "Romantic dinner spot for anniversary" or "Quiet workspace with parking"

3. **CaminoJourney** - Multi-stop journey planning
   - Use for: Planning trips with multiple destinations
   - Features: Route optimization, time estimation, transport modes
   - Best for: "Plan a day of sightseeing" or "Optimize delivery route"

**Guidelines for Location Intelligence:**

1. **Understand Context**: Consider the user's intent, time constraints, and preferences
2. **Be Specific**: Use detailed queries to get better results
3. **Consider Proximity**: Factor in distances and travel times
4. **Provide Options**: Offer multiple choices when appropriate
5. **Add Value**: Include helpful details like hours, ratings, or tips

**Reasoning Process:**
- Thought: Analyze what information is needed
- Action: Choose and execute the appropriate tool
- Observation: Evaluate the results
- Iterate as needed to provide comprehensive answers

**Example Workflows:**

**Simple Search:**
User: "Find Italian restaurants nearby"
→ Use CaminoSearch with query "Italian restaurants"

**Context-Aware Search:**
User: "I need a quiet place to study with good coffee"
→ Use CaminoQuery with context about studying needs

**Journey Planning:**
User: "Plan a tourist day visiting major attractions"
→ Use CaminoJourney to optimize the route between attractions

Remember to:
- Provide clear, actionable recommendations
- Include relevant details (distance, time, cost estimates)
- Explain your reasoning when making suggestions
- Ask for clarification if the request is ambiguous
"""
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]),
    name="location_agent"
)

print("🤖 Location-aware ReAct agent created successfully!")

🤖 Location-aware ReAct agent created successfully!


### Test the Location Agent

In [27]:
from langchain.schema import HumanMessage
from IPython.display import Markdown

# Test the agent with a simple request
print("🧪 Test 1: Simple Location Search\n")

inputs = {
    "messages": [
        HumanMessage(content="Find the best ramen restaurants in Manhattan")
    ]
}

# Stream the agent's response
for chunk in location_agent.stream(inputs, stream_mode="values"):
    message = chunk["messages"][-1]
    if hasattr(message, 'pretty_print'):
        message.pretty_print()

# Display the final answer
final_message = chunk["messages"][-1]
if hasattr(final_message, 'content'):
    display(Markdown(final_message.content))

🧪 Test 1: Simple Location Search

================================ Human Message =================================

Find the best ramen restaurants in Manhattan
================================== Ai Message ==================================
Name: location_agent

[{'text': "I'd be happy to find the best ramen restaurants in Manhattan for you! Let me search for that information.", 'type': 'text'}, {'id': 'toolu_01XfaWSq85vEQXgp1At3UMoX', 'input': {'query': 'best ramen restaurants in Manhattan, New York', 'limit': 10, 'ai_ranking': True}, 'name': 'camino_query', 'type': 'tool_use'}]
Tool Calls:
  camino_query (toolu_01XfaWSq85vEQXgp1At3UMoX)
 Call ID: toolu_01XfaWSq85vEQXgp1At3UMoX
  Args:
    query: best ramen restaurants in Manhattan, New York
    limit: 10
    ai_ranking: True
================================= Tool Message =================================
Name: camino_query

{"results": [{"name": "Mr. Keke", "lat": 40.7290829, "lon": -73.990007, "distance": null, "type": "restaurant"

# Best Ramen Restaurants in Manhattan

Based on my search, here are the top ramen restaurants in Manhattan, ranked by relevance:

1. **Mr. Keke** - Located in East Village (40.729°N, 73.990°W)
   - Highly rated authentic ramen experience

2. **Ippudo** - Multiple locations:
   - East Village location (40.731°N, 73.990°W)
   - Midtown location (40.764°N, 73.987°W)
   - Third location (40.756°N, 73.980°W)
   - Known for their famous Hakata-style tonkotsu ramen

3. **Toribro Ramen** - Midtown (40.764°N, 73.988°W)
   - Specializing in authentic Japanese ramen

4. **Totto Ramen** - Hell's Kitchen area (40.765°N, 73.991°W)
   - Known for their paitan (chicken-based) ramen

5. **Ajisen Ramen** - Chinatown (40.714°N, 73.999°W)
   - Popular chain offering various ramen styles

6. **Karakatta** - East Village (40.729°N, 73.999°W)
   - Known for spicy ramen options

7. **Meijin Ramen** - Upper East Side (40.775°N, 73.954°W)
   - Quality ramen in a less crowded neighborhood

8. **Rai Rai Ken** - East Village (40.729°N, 73.986°W)
   - One of the original ramen shops in Manhattan

These restaurants represent some of the best ramen options across different Manhattan neighborhoods. The East Village and Midtown areas appear to have the highest concentration of top-rated ramen restaurants. Ippudo is particularly notable with three locations throughout Manhattan, suggesting its popularity among ramen enthusiasts.

Would you like more specific information about any of these restaurants, such as their specialties or recommended dishes?

## Conclusion & Next Steps 🎯

Congratulations! You've learned how to build sophisticated location-aware agents using LangChain and Camino AI.

### What You've Accomplished:
- ✅ Set up Camino AI tools with LangChain
- ✅ Built ReAct agents for location reasoning
- ✅ Created agents with memory for personalization
- ✅ Combined multiple tools for comprehensive intelligence

### Key Takeaways:
1. **Location Intelligence Matters**: Physical world understanding is crucial for many AI applications
2. **Tool Synergy**: Combining Camino with other tools creates powerful agents
3. **Context is King**: The more context you provide, the better the results
4. **Memory Enhances UX**: Agents that remember preferences provide better experiences
5. **Production Readiness**: Error handling, caching, and monitoring are essential

### Next Steps:
1. **Experiment**: Try different agent architectures and prompts
2. **Combine Tools**: Add web search, databases, or other APIs
3. **Build Applications**: Create domain-specific agents for your use case
4. **Deploy**: Use these patterns in production applications
5. **Share**: Contribute to the community with your innovations

### Resources:
- 📚 [Camino AI Documentation](https://api.getcamino.ai/docs)
- 🔧 [LangChain Documentation](https://python.langchain.com/)
- 🎮 [Interactive Playground](https://app.getcamino.ai/playground)

### Advanced Topics to Explore:
- **Multi-Agent Systems**: Coordinate multiple specialized location agents
- **Streaming Responses**: Implement real-time location updates
- **Custom Tools**: Build domain-specific location tools
- **Fine-Tuning**: Optimize prompts for specific industries
- **Hybrid Intelligence**: Combine with computer vision for visual location understanding

---

**Ready to build production location-aware AI agents?** Start with Camino AI and LangChain to give your agents the power to understand and navigate the physical world! 🚀

Happy building! 🏗️